In [1]:
import os
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tensorflow import keras

In [2]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

In [3]:
root_dir = './cat_train_images_2'
filenames = sorted(get_file_list(root_dir))

TRAIN_SAMPLES = len(filenames)
NUM_CLASSES = 7
IMG_WIDTH, IMG_HEIGHT = 224, 224
batch_size=16

preprocess_input = applications.densenet.preprocess_input

input_model = "./densenet201-epoch0_2.h5"
output_model = "./densenet201-epoch1_2.h5"

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2)

In [5]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                   target_size=(IMG_WIDTH,
                                                               IMG_HEIGHT),
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=12345,
                                                   class_mode='categorical')

Found 35 images belonging to 7 classes.


In [6]:
model_finetuned = keras.models.load_model(input_model)
model_finetuned.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
densenet201 (Functional)     (None, 7, 7, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                122944    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 455       
Total params: 18,445,383
Trainable params: 123,399
Non-trainable params: 18,321,984
____________________________________

In [7]:
model_finetuned.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(0.001),
                       metrics=['acc'])

model_finetuned.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / batch_size),
    epochs=8
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/8
3/3 [==============================] - 4s 1s/step - loss: 2.6028 - acc: 0.0857
Epoch 2/8
3/3 [==============================] - 4s 1s/step - loss: 2.6006 - acc: 0.2286
Epoch 3/8
3/3 [==============================] - 4s 1s/step - loss: 2.3290 - acc: 0.2286
Epoch 4/8
3/3 [==============================] - 2s 783ms/step - loss: 2.0034 - acc: 0.3429
Epoch 5/8
3/3 [==============================] - 2s 778ms/step - loss: 1.6912 - acc: 0.3429
Epoch 6/8
3/3 [==============================] - 2s 787ms/step - loss: 1.5982 - acc: 0.4286
Epoch 7/8
3/3 [==============================] - 4s 1s/step - loss: 1.4861 - acc: 0.4286
Epoch 8/8
3/3 [==============================] - 2s 699ms/step - loss: 1.3632 - acc: 0.4571


In [8]:
model_finetuned.save(output_model)